In [1]:
# run this by installing required packages in local machine
# google colab doesn't work well with VideoStream package
# import the necessary packages
import numpy as np
import argparse
import imutils
import time
import cv2
import os

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream

In [2]:
# using the same function used in face mask detection in detecting from an image
def detect_and_predict_mask(frame, faceNet, maskNet):
    # construct a blob from the image to pass to the network
    # using standard weights for the face detection model for image preprocessing
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
        (104.0, 177.0, 123.0))

    # obtain the face detections by passing the blob through the network 
    faceNet.setInput(blob)
    detections = faceNet.forward()

    # initialize our list of faces, their corresponding locations,
    # and the list of predictions from our face mask network
    faces = []
    locs = []
    preds = []

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract only confident detections using the confidence/probability 
        # associated with the detection
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence 0.5 or input variable
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # making sure bounding box is inside the dimensions of the image
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face region of interest(ROI), convert it from BGR to RGB channel
            # ordering, resize it to 224x224, and preprocessing it to be compatible with
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            
            # add the face and bounding boxes to their respective
            # lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))

    # only make a predictions if at least one face was detected
    if len(faces) > 0:
        # for faster inference we'll make batch predictions on *all*
        # faces at the same time rather than one-by-one predictions
        # in the above `for` loop
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    # return face locations and their corresponding locations as a tuple
    return (locs, preds)

In [3]:
# loading the pretrained face detection OpenCV2 caffe model
print("loading pre-trained openCV face detector model...")
# model architecture (i.e., the layers) file path
prototxtPath = os.path.sep.join(["face_detector", "deploy.prototxt"])
# model weights file path
weightsPath = os.path.sep.join(["face_detector","res10_300x300_ssd_iter_140000.caffemodel"])
# loading the pretrained face detection model using above architecture ad weights
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask classifier built using dev_train_mask_detector file
print("loading the face mask classifier model...")
maskNet = load_model("mask_detector.model")

# initialize the video stream and allow the camera sensor to start
print("starting the video stream windw to track faces and clasify them")
vs = VideoStream(src=0).start()
time.sleep(2.0)

# loop over the frames from the video stream
while True:
    # grabing a frame from the video stream and resizing the frame to 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    # detect faces in the frame and determine if they are wearing a
    # face mask or not
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    # loop over the detected face locations and their corresponding
    # locations
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        # find the class and associated colour to use for the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # include the probability of prediction in the label of the bounding box
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(frame, label, (startX, startY - 10),
        cv2.FONT_HERSHEY_COMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame including the bounding box and label
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

loading pre-trained openCV face detector model...
loading the face mask classifier model...
starting the video stream windw to track faces and clasify them
